# 🔍 Frequent Itemset Mining

Discover patterns in flight delays using association rule mining.

**Objectives:**
- Find frequent route/carrier combinations that experience delays
- Identify cascade delay patterns
- Generate actionable association rules


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Try to import mlxtend for association rules
try:
    from mlxtend.frequent_patterns import apriori, association_rules
    from mlxtend.preprocessing import TransactionEncoder
    MLXTEND_AVAILABLE = True
except ImportError:
    print("⚠️ mlxtend not installed. Install with: pip install mlxtend")
    MLXTEND_AVAILABLE = False

COLORS = {'primary': '#2E86AB', 'secondary': '#A23B72', 'warning': '#F18F01'}
print("✓ Libraries imported")


In [ ]:
# Load data
df = pd.read_csv('../data/processed/flights_cleaned.csv')

# Create transaction items for delayed flights
delayed_flights = df[df['is_delayed'] == 1].copy()
print(f"Total delayed flights: {len(delayed_flights):,}")

# Create item columns
route_col = [c for c in df.columns if 'route' in c.lower()]
carrier_col = [c for c in df.columns if 'airline' in c.lower() or 'carrier' in c.lower()]

items_df = pd.DataFrame()
if route_col:
    items_df['route'] = 'ROUTE_' + delayed_flights[route_col[0]].astype(str)
if carrier_col:
    items_df['carrier'] = 'CARRIER_' + delayed_flights[carrier_col[0]].astype(str)

# Add time-based items
if 'hour' in df.columns:
    items_df['time'] = delayed_flights['hour'].apply(lambda x: f"HOUR_{x:02d}" if pd.notna(x) else None)
if 'day_of_week' in df.columns:
    days = ['MON', 'TUE', 'WED', 'THU', 'FRI', 'SAT', 'SUN']
    items_df['day'] = delayed_flights['day_of_week'].apply(lambda x: f"DAY_{days[int(x)]}" if pd.notna(x) else None)

print(f"Created {len(items_df.columns)} item categories")
display(items_df.head())


## Association Rule Mining


In [ ]:
if MLXTEND_AVAILABLE and len(items_df) > 0:
    # Convert to transactions
    transactions = items_df.values.tolist()
    transactions = [[item for item in tx if pd.notna(item)] for tx in transactions]
    
    # One-hot encode
    te = TransactionEncoder()
    te_array = te.fit_transform(transactions)
    df_encoded = pd.DataFrame(te_array, columns=te.columns_)
    
    # Find frequent itemsets
    frequent_itemsets = apriori(df_encoded, min_support=0.01, use_colnames=True)
    frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(len)
    
    print(f"Found {len(frequent_itemsets)} frequent itemsets")
    display(frequent_itemsets.sort_values('support', ascending=False).head(15))
    
    # Generate association rules
    if len(frequent_itemsets) > 0:
        rules = association_rules(frequent_itemsets, metric='lift', min_threshold=1.0)
        rules = rules.sort_values('lift', ascending=False)
        print(f"\nFound {len(rules)} association rules with lift > 1.0")
        
        if len(rules) > 0:
            # Display top rules
            display(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head(15))
            
            # Visualize rules
            fig, ax = plt.subplots(figsize=(10, 6))
            ax.scatter(rules['support'], rules['confidence'], c=rules['lift'], 
                      cmap='viridis', alpha=0.6, s=50)
            plt.colorbar(ax.collections[0], label='Lift')
            ax.set_xlabel('Support')
            ax.set_ylabel('Confidence')
            ax.set_title('Association Rules: Support vs Confidence')
            plt.savefig('../reports/figures/association_rules.png', dpi=150)
            plt.show()
else:
    print("Performing basic frequency analysis instead...")
    # Simple frequency analysis as fallback
    for col in items_df.columns:
        print(f"\n{col.upper()} frequency in delayed flights:")
        print(items_df[col].value_counts().head(10))

print("\n✓ Itemset mining complete!")
